# Set up plotting in the notebook

In [ ]:
%matplotlib inline

In [ ]:
# %matplotlib

# Imports

In [ ]:
from __future__ import division

In [ ]:
from bokeh.charts import Scatter, output_file, show
from bokeh.sampledata.autompg import autompg as df
from bokeh.models import HoverTool
from bokeh.models import GlyphRenderer
from bokeh.io import output_notebook

In [ ]:
# output_notebook()

In [ ]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster.bicluster import SpectralBiclustering
from sklearn.metrics import consensus_score

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, normalize, scale

from sklearn.feature_selection import SelectKBest

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.set_context("poster")

# Load data

In [ ]:
data = pd.read_excel("/home/gus/MEGAsync/zim/main/BCH/Projects/Naresh/RNAseq/genes_log3WH.xlsx")

In [ ]:
data.head()

In [ ]:
data_ = data[["gene_id","1WH1_FPKM","1WK1_FPKM","3WH1_FPKM","3WK1_FPKM","12WH2_FPKM","12WK2_FPKM",]]

# Transpose the data so we are PCA'ing the conditions

In [ ]:
data_.head()

In [ ]:
dataT_ = data_.T.drop('gene_id',axis=0)

In [ ]:
dataT_

# Make map from gene_short_name to gene_id

In [ ]:
to_gene_id = {}

for i in range(len(data)):
    to_gene_id[data.iloc[i,4]] = data.iloc[i,3]

In [ ]:
to_gene_id['Stat1']

## Looks like we have dupplicated values for gene_short_name

In [ ]:
len(data.gene_short_name.unique())

In [ ]:
len(data)

In [ ]:
dup_short_name = data[data.gene_short_name.duplicated(keep=False)].copy()

In [ ]:
dup_short_name['gene_short_name'] = dup_short_name.gene_short_name.astype(str)

In [ ]:
dup_short_name = dup_short_name.sort_values('gene_short_name')

In [ ]:
len(dup_short_name)

In [ ]:
dup_short_name

# Get correlations

In [ ]:
data_T = data_.set_index('gene_id').T

In [ ]:
data_T.head()

In [ ]:
r = data_T.corr()

In [ ]:
len(r)

In [ ]:
r.iloc[:6,:6]

In [ ]:
stat1 = to_gene_id['Stat1']

In [ ]:
stat1_list = r[stat1_list]

In [ ]:
stat1_list.sort_values(acending=)